# Gráficos de series de tiempo normalizadas

Importación de librerías.

In [ ]:
import sys
import os
from functools import reduce

project_path = os.path.abspath('..')
sys.path.insert(1, project_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade

from src.utils import get_project_root
from src.data.make_dataset import get_minma_data

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

Definición de funciones para obtención de gráficos.

In [ ]:
def stand(df):
    '''
    Returns a dataframe with all its columns standarize ( (x - mu) / sigma ) 
    '''
    stand_df = df.copy()

    for col_name in stand_df:
        stand_df[col_name] = ( stand_df[col_name] - stand_df[col_name].mean() ) / stand_df[col_name].std()
        
    return stand_df

In [ ]:
def peak_df2(station, threshold, params):
    df = get_minma_data(params, station, from_last=from_last, n_cols=len(params))
    cols_to_drop = list(df.filter(regex='(?<!no validados)_.*')) 
    df = df.drop(cols_to_drop, axis=1)
    df = df.reset_index().rename(columns={"Registros no validados_SO2": "SO2", "Registros no validados_NO2": "NO2", "Registros no validados_NO": "NO", "Registros no validados_NOX": "NOX", "Registros no validados_O3": "O3", "Registros no validados_CO": "CO", "Registros no validados_MP10": "MP10","Registros no validados_MP25": "MP25", "index": "date"})
    
    mask_peaks = df["SO2"] >= threshold
    peaks_df = df[mask_peaks]
    
    peaks_df['date'] = pd.to_datetime(peaks_df['date']).dt.date # date_time to dates
    crit_days = peaks_df["date"].unique() # unique peak days array
    
    values = [ df["date"].dt.date == day for day in crit_days ] # get a mask for each crit day
    mask_crit_days = reduce(lambda x,y: np.logical_or(x,y), values)
    
    # filtering df with the mask of critical days
    peaks_df = df[mask_crit_days]
       
    return peaks_df, crit_days

In [ ]:
def peak_df3(df, station, threshold):
    '''
    df with date on its index
    '''
    
    mask_peaks = df["SO2"] >= threshold
    peaks_df = df[mask_peaks]
    
    peaks_df = peaks_df.set_index( peaks_df.index.date ) # datetime to date to the index
    crit_days = peaks_df.index.unique() # unique peak days array
    
    values = [ df.index.date == day for day in crit_days ] # get a mask for each crit day
    mask_crit_days = reduce(lambda x,y: np.logical_or(x,y), values)
    
    # filtering df with the mask of critical days
    peaks_df = df[mask_crit_days]
       
    return peaks_df, crit_days

In [ ]:
def generate_var_hour_df( peaks_df, var_name ):
    df_var = peaks_df[[var_name]]
    df_var["Time"] = df_var.index.date
    df_var["Hour"] = df_var.index.hour
    df_var = df_var.pivot(index='Hour', columns='Time', values=var_name)
    return df_var

In [ ]:
def all_var_plot(all_peaks_df, all_var_names, station):
    '''
    Grafica las series de tiempo del dataframe ingresado. Establece también los límites de T_min y T_max 
    normalizados en el gráfico.
    '''
    
    sns.set_style('darkgrid')
    sns.set_context('talk')
    
    total_len_index = len(all_peaks_df) - 2 # without T10 and T40
    for c in range(len(all_peaks_df[0].columns)): # for each column (day)
        
        plt.figure(figsize=(18, 10), dpi=80)
        for df_idx in range(total_len_index): # for each index_df (df_variable)
            df = all_peaks_df[df_idx]
            plt.plot(df.index, df.iloc[:,[c]], label=all_var_names[df_idx]) # plot with variable legend
            
        T10 = all_peaks_df[-2]
        T40 = all_peaks_df[-1]
        
        plt.hlines(T10.iloc[:,[c]].min(), xmin=0, xmax=23, label='T10', linestyles='dashed', colors='k')
        plt.hlines(T10.iloc[:,[c]].max(), xmin=0, xmax=23, linestyles='dashed', colors='k')
        
        plt.hlines(T40.iloc[:,[c]].min(), xmin=0, xmax=23, label='T40', linestyles='dashed', colors='b')
        plt.hlines(T40.iloc[:,[c]].max(), xmin=0, xmax=23, linestyles='dashed', colors='b')

        plt.text(23, T10.iloc[:,[c]].max(),'max')
        plt.text(23, T10.iloc[:,[c]].min(),'min')
        plt.legend()

        plt.title(f'{station.capitalize()} - Dia {df.iloc[:,c].name}')
        plt.show()

## Estación Quintero

Análsis de datos de la estación Quintero.

Manejo de datos de la estación Quintero.

In [ ]:
# parametros a analizar por estacion
params = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']
params_quintero = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']
params_quintero_2 = ['velviento', 'dirviento', 'T04', 'T10']

params_maitenes = ['SO2','NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25']
params_ventanas = ['SO2','NO2', 'NO', 'NOX', 'O3', 'MP10', 'MP25']
from_last = '5y' # ventanas de tiempo

In [ ]:
meteo_df = pd.read_excel(get_project_root() / 'data' / 'raw' / 'Torre Meteo Codelco Ventanas.xlsx')
meteo_df = meteo_df.set_index('date')

In [ ]:
meteo_df.head()

In [ ]:
# estableciendo conjuntos de datos para trabajar
params_ = params
station = 'quintero'
get_minma_data(params_, station, from_last=from_last)

quintero_df = get_minma_data(params, station, from_last=from_last)
cols_to_drop = list(quintero_df.filter(regex='(?<!no validados)_.*'))
quintero_df = quintero_df.drop(cols_to_drop, axis=1)
quintero_df = quintero_df.rename(columns={"Registros no validados_SO2": "SO2", "Registros no validados_NO2": "NO2", "Registros no validados_NO": "NO", "Registros no validados_NOX": "NOX", "Registros no validados_O3": "O3", "Registros no validados_CO": "CO", "Registros no validados_MP10": "MP10","Registros no validados_MP25": "MP25", "index": "date"})

quintero_df2 = get_minma_data(['velviento', 'dirviento'], station, from_last=from_last, n_cols = 3)
quintero_df2 = quintero_df2.rename(columns={"Unnamed: 2_velviento": "VelViento", "Unnamed: 2_dirviento": "DirViento"})#, "Unnamed: 2_T04": "T_04", "Unnamed: 2_T10": "T_10"})

# Union de datos
quintero_df = quintero_df.join(quintero_df2)
quintero_df = quintero_df.join(meteo_df)

# conjunto de datos estandarizados
quintero_stand_df = stand(quintero_df)

# Obtención de días críticos
quintero_peaks_df, quintero_crit_days = peak_df3( quintero_df, 'quintero', threshold=133 )
quintero_stand_peaks_df = stand( quintero_peaks_df )

In [ ]:
# non standarized
quintero_SO2 = generate_var_hour_df( quintero_peaks_df, "SO2" )
quintero_NO2 = generate_var_hour_df( quintero_peaks_df, "NO2" )
quintero_NO = generate_var_hour_df( quintero_peaks_df, "NO" )
quintero_NOX = generate_var_hour_df( quintero_peaks_df, "NOX" )
quintero_O3 = generate_var_hour_df( quintero_peaks_df, "O3" )
quintero_CO = generate_var_hour_df( quintero_peaks_df, "CO" )
quintero_MP10 = generate_var_hour_df( quintero_peaks_df, "MP10" )
quintero_MP25 = generate_var_hour_df( quintero_peaks_df, "MP25" )
quintero_T10 = generate_var_hour_df( quintero_peaks_df, "T_20" )
quintero_T40 = generate_var_hour_df( quintero_peaks_df, "T_40" )

# standarized
quintero_stand_SO2 = generate_var_hour_df( quintero_stand_peaks_df, "SO2" )
quintero_stand_NO2 = generate_var_hour_df( quintero_stand_peaks_df, "NO2" )
quintero_stand_NO = generate_var_hour_df( quintero_stand_peaks_df, "NO" )
quintero_stand_NOX = generate_var_hour_df( quintero_stand_peaks_df, "NOX" )
quintero_stand_O3 = generate_var_hour_df( quintero_stand_peaks_df, "O3" )
quintero_stand_CO = generate_var_hour_df( quintero_stand_peaks_df, "CO" )
quintero_stand_MP10 = generate_var_hour_df( quintero_stand_peaks_df, "MP10" )
quintero_stand_MP25 = generate_var_hour_df( quintero_stand_peaks_df, "MP25" )
quintero_stand_T10 = generate_var_hour_df( quintero_stand_peaks_df, "T_20" )
quintero_stand_T40 = generate_var_hour_df( quintero_stand_peaks_df, "T_40" )


quintero_all_peaks_df = [quintero_SO2,quintero_NO2,quintero_NO, quintero_NOX, quintero_O3, quintero_CO, quintero_MP10, quintero_MP25, quintero_T10, quintero_T40]
quintero_stand_all_peaks_df = [quintero_stand_SO2 ,quintero_stand_NO2, quintero_stand_NO, quintero_stand_NOX, quintero_stand_O3, quintero_stand_CO, quintero_stand_MP10, quintero_stand_MP25, quintero_stand_T10, quintero_stand_T40]
quintero_all_var_names = ['SO2', 'NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25', 'T10', 'T40']

Serie de tiempo de todas las variables para días peak con datos estandarizados.

In [ ]:
# Standarized
all_var_plot( quintero_stand_all_peaks_df, quintero_all_var_names, 'quintero')

Serie de tiempo de todas las variables para días peak con datos no estandarizados.

In [ ]:
# Non standarized
all_var_plot( quintero_all_peaks_df, quintero_all_var_names, 'quintero' )

<!-- Histograma de frecuencia de peaks por cada hora para la estación Quintero. -->

## Estación Los Maitenes

Análisis de datos de estación Los Maitenes.

Manejo de datos de la estación Los Maitenes.

In [ ]:
# estableciendo conjuntos de datos para trabajar.
params_ = params
station = 'maitenes'
get_minma_data(params_, station, from_last=from_last)

maitenes_df = get_minma_data(params, station, from_last=from_last)
cols_to_drop = list(maitenes_df.filter(regex='(?<!no validados)_.*'))
maitenes_df = maitenes_df.drop(cols_to_drop, axis=1)
maitenes_df = maitenes_df.rename(columns={"Registros no validados_SO2": "SO2", "Registros no validados_NO2": "NO2", "Registros no validados_NO": "NO", "Registros no validados_NOX": "NOX", "Registros no validados_O3": "O3", "Registros no validados_CO": "CO", "Registros no validados_MP10": "MP10","Registros no validados_MP25": "MP25", "index": "date"})

maitenes_df2 = get_minma_data(['velviento', 'dirviento'], station, from_last=from_last, n_cols = 3)
maitenes_df2 = maitenes_df2.rename(columns={"Unnamed: 2_velviento": "VelViento", "Unnamed: 2_dirviento": "DirViento"})#, "Unnamed: 2_T04": "T_04", "Unnamed: 2_T10": "T_10"})

# Conjunto de datos no estandarizado
maitenes_df = maitenes_df.join(maitenes_df2)
maitenes_df = maitenes_df.join(meteo_df)

# Conjunto de datos estandarizado
maitenes_stand_df = stand(maitenes_df)

# Obtención de días críticos
maitenes_peaks_df, maitenes_crit_days = peak_df3( maitenes_df, 'maitenes', threshold=133 )
maitenes_stand_peaks_df = stand( maitenes_peaks_df )

In [ ]:
# non standarized
maitenes_SO2 = generate_var_hour_df( maitenes_peaks_df, "SO2" )
maitenes_NO2 = generate_var_hour_df( maitenes_peaks_df, "NO2" )
maitenes_NO = generate_var_hour_df( maitenes_peaks_df, "NO" )
maitenes_NOX = generate_var_hour_df( maitenes_peaks_df, "NOX" )
maitenes_O3 = generate_var_hour_df( maitenes_peaks_df, "O3" )
maitenes_CO = generate_var_hour_df( maitenes_peaks_df, "CO" )
maitenes_MP10 = generate_var_hour_df( maitenes_peaks_df, "MP10" )
maitenes_MP25 = generate_var_hour_df( maitenes_peaks_df, "MP25" )
maitenes_T10 = generate_var_hour_df( maitenes_peaks_df, "T_20" )
maitenes_T40 = generate_var_hour_df( maitenes_peaks_df, "T_40" )

# standarized
maitenes_stand_SO2 = generate_var_hour_df( maitenes_stand_peaks_df, "SO2" )
maitenes_stand_NO2 = generate_var_hour_df( maitenes_stand_peaks_df, "NO2" )
maitenes_stand_NO = generate_var_hour_df( maitenes_stand_peaks_df, "NO" )
maitenes_stand_NOX = generate_var_hour_df( maitenes_stand_peaks_df, "NOX" )
maitenes_stand_O3 = generate_var_hour_df( maitenes_stand_peaks_df, "O3" )
maitenes_stand_CO = generate_var_hour_df( maitenes_stand_peaks_df, "CO" )
maitenes_stand_MP10 = generate_var_hour_df( maitenes_stand_peaks_df, "MP10" )
maitenes_stand_MP25 = generate_var_hour_df( maitenes_stand_peaks_df, "MP25" )
maitenes_stand_T10 = generate_var_hour_df( maitenes_stand_peaks_df, "T_20" )
maitenes_stand_T40 = generate_var_hour_df( maitenes_stand_peaks_df, "T_40" )


maitenes_all_peaks_df = [maitenes_SO2,quintero_NO2,maitenes_NO, maitenes_NOX, maitenes_O3, maitenes_CO, maitenes_MP10, maitenes_MP25, maitenes_T10, maitenes_T40]
maitenes_stand_all_peaks_df = [maitenes_stand_SO2 ,maitenes_stand_NO2, maitenes_stand_NO, maitenes_stand_NOX, maitenes_stand_O3, maitenes_stand_CO, maitenes_stand_MP10, maitenes_stand_MP25, maitenes_stand_T10, maitenes_stand_T40]
maitenes_all_var_names = ['SO2', 'NO2', 'NO', 'NOX', 'O3', 'CO', 'MP10', 'MP25', 'T10', 'T40']

Gráfico de series de tiempo normalizadas para los días con peaks en la estación Los Maitenes.

In [ ]:
all_var_plot( maitenes_stand_all_peaks_df, maitenes_all_var_names, 'maitenes')

Gráfico de series de tiempo no normalizadas para los días con peaks en la estación Los Maitenes.

In [ ]:
all_var_plot( maitenes_all_peaks_df, maitenes_all_var_names, 'maitenes' )